In [2]:
import numpy as np
import keras
from keras import callbacks
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [3]:
img_row = 108
img_col = 108
path = "E:/database108/"
batch_size = 64
output_classes = 24

In [4]:
import random
import pandas as pd
x_train_ori = []
y_train_ori = []
x_test_ori = []
y_test_ori = []
for i in range(516):
    print(i)
    file = pd.read_csv(path + "database_{}.csv".format(i), dtype=str)
    array = file.values
    array = list(array)
    train_test = random.sample(range(100), 60)
    
    for j in train_test[:50]:
        sample = list(array[j])
        lst = []
        for x in sample[1:11665]:
            if type(x) == str:
                x = "".join(x.split())
                lst.append(float(x))
            elif type(x) == float:
                lst.append(x)
            else:
                print(type(x))
        x_train_ori.append(lst)
        y_train_ori.append(sample[0])
        
    for l in train_test[50:]:
        sample = list(array[l])
        lst = []
        for x in sample[1:11665]:
            if type(x) == str:
                x = "".join(x.split())
                lst.append(float(x))
            elif type(x) == float:
                lst.append(x)
            else:
                print(type(x))
        x_test_ori.append(lst)
        y_test_ori.append(sample[0])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [5]:
print(np.shape(x_train_ori))
print(np.shape(x_test_ori))
print(np.shape(y_train_ori))
print(np.shape(y_test_ori))

(25800, 11664)
(5160, 11664)
(25800,)
(5160,)


In [6]:
def y_convert(array, flag=0):
    y = []
    for i in array: # i is the name of each sample:
        name = i.split("_")
        name = name[1:5]
        if flag == 0:
            # category of appearance of a note
            category = np.zeros(24, dtype=int)
            for j in name:
                if j != "x":
                    num = int(j)
                    category[num-60] = 1
            y.append(list(category))
    return y

In [7]:
lst_1 = []
lst_2 = []

for i in y_train_ori:
    label = "".join(i.split())
    lst_1.append(label)
for j in y_test_ori:
    label = "".join(j.split())
    lst_2.append(label)
    
y_train_ori = y_convert(lst_1)
y_test_ori = y_convert(lst_2)

In [8]:
y_train_ori = np.array(y_train_ori)
y_test_ori = np.array(y_test_ori)
x_train_ori = np.array(x_train_ori)
x_test_ori = np.array(x_test_ori)

In [9]:
x_train_ori = x_train_ori.flatten()
x_train_ori = x_train_ori.reshape(25800, img_row, img_col, 1)
x_test_ori = x_test_ori.flatten()
x_test_ori = x_test_ori.reshape(5160, img_row, img_col, 1)

In [10]:
print(x_train_ori.shape)
input_shape = (108, 108, 1)

(25800, 108, 108, 1)


In [17]:
def build_model_E():
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #third convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))

    #forth convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #fifth convulutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.3))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(64, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
        
    #second FC layer
    model.add(Dense(128, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    
    #third FC layer
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    #output layer
    model.add(Dense(24, activation="sigmoid", kernel_initializer="uniform"))
    
    model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])
    return model

In [18]:
classifier_E = build_model_E()
hist_E = classifier_E.fit(x_train_ori, y_train_ori, batch_size=64, nb_epoch=100, verbose=1,
                          validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_ep

Train on 25800 samples, validate on 5160 samples
Epoch 1/100
25800/25800 [==============================] - 22s 871us/step - loss: 0.0641 - acc: 0.3021 - val_loss: 0.0017 - val_acc: 0.2936
Epoch 2/100
25800/25800 [==============================] - 18s 689us/step - loss: 0.0046 - acc: 0.2790 - val_loss: 8.6305e-04 - val_acc: 0.2329
Epoch 3/100
25800/25800 [==============================] - 18s 690us/step - loss: 0.0026 - acc: 0.2753 - val_loss: 4.0472e-04 - val_acc: 0.4172
Epoch 4/100
25800/25800 [==============================] - 18s 691us/step - loss: 0.0018 - acc: 0.2717 - val_loss: 2.0647e-04 - val_acc: 0.1516
Epoch 5/100
25800/25800 [==============================] - 18s 692us/step - loss: 0.0016 - acc: 0.2740 - val_loss: 5.3611e-05 - val_acc: 0.2921
Epoch 6/100
25800/25800 [==============================] - 18s 691us/step - loss: 0.0016 - acc: 0.2770 - val_loss: 8.9945e-05 - val_acc: 0.2198
Epoch 7/100
25800/25800 [==============================] - 18s 691us/step - loss: 0.0012 - 

25800/25800 [==============================] - 18s 714us/step - loss: 3.2281e-04 - acc: 0.7151 - val_loss: 3.4271e-08 - val_acc: 0.8438
Epoch 57/100
25800/25800 [==============================] - 18s 710us/step - loss: 2.0581e-04 - acc: 0.7233 - val_loss: 5.3592e-08 - val_acc: 0.9178
Epoch 58/100
25800/25800 [==============================] - 18s 692us/step - loss: 2.5602e-04 - acc: 0.7378 - val_loss: 7.4405e-05 - val_acc: 0.6797
Epoch 59/100
25800/25800 [==============================] - 18s 692us/step - loss: 4.0609e-04 - acc: 0.7276 - val_loss: 5.4883e-08 - val_acc: 0.8337
Epoch 60/100
25800/25800 [==============================] - 18s 694us/step - loss: 2.8725e-04 - acc: 0.7338 - val_loss: 0.2850 - val_acc: 0.0537
Epoch 61/100
25800/25800 [==============================] - 18s 694us/step - loss: 6.6636e-04 - acc: 0.7414 - val_loss: 8.4885e-07 - val_acc: 0.7713
Epoch 62/100
25800/25800 [==============================] - 18s 696us/step - loss: 3.0758e-04 - acc: 0.7503 - val_loss: 0.2

In [21]:
fname = "weight-25800-CNN_E_good.hdf5"
classifier_E.save_weights(fname, overwrite=True)

In [20]:
classifier_E = build_model_E()
hist_E = classifier_E.fit(x_train_ori, y_train_ori, batch_size=64, nb_epoch=200, verbose=1,
                          validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_ep

Train on 25800 samples, validate on 5160 samples
Epoch 1/200
25800/25800 [==============================] - 20s 775us/step - loss: 0.0636 - acc: 0.2650 - val_loss: 0.0042 - val_acc: 0.1512
Epoch 2/200
25800/25800 [==============================] - 18s 688us/step - loss: 0.0045 - acc: 0.2765 - val_loss: 0.0020 - val_acc: 0.2777
Epoch 3/200
25800/25800 [==============================] - 18s 688us/step - loss: 0.0027 - acc: 0.2568 - val_loss: 4.5560e-04 - val_acc: 0.4023
Epoch 4/200
25800/25800 [==============================] - 18s 695us/step - loss: 0.0018 - acc: 0.2676 - val_loss: 0.0076 - val_acc: 0.3952
Epoch 5/200
25800/25800 [==============================] - 18s 693us/step - loss: 0.0015 - acc: 0.2613 - val_loss: 6.9962e-04 - val_acc: 0.2533
Epoch 6/200
25800/25800 [==============================] - 18s 703us/step - loss: 0.0015 - acc: 0.2622 - val_loss: 0.0170 - val_acc: 0.3587
Epoch 7/200
25800/25800 [==============================] - 18s 687us/step - loss: 0.0020 - acc: 0.2692 

25800/25800 [==============================] - 19s 742us/step - loss: 3.4611e-04 - acc: 0.6762 - val_loss: 7.3281e-06 - val_acc: 0.7990
Epoch 57/200
25800/25800 [==============================] - 19s 746us/step - loss: 4.5596e-04 - acc: 0.6823 - val_loss: 2.7369e-06 - val_acc: 0.8847
Epoch 58/200
25800/25800 [==============================] - 19s 746us/step - loss: 1.9902e-04 - acc: 0.7125 - val_loss: 3.1915e-06 - val_acc: 0.8421
Epoch 59/200
25800/25800 [==============================] - 19s 735us/step - loss: 2.5181e-04 - acc: 0.6979 - val_loss: 7.6767e-07 - val_acc: 0.8021
Epoch 60/200
25800/25800 [==============================] - 19s 726us/step - loss: 2.7619e-04 - acc: 0.6906 - val_loss: 8.1812e-08 - val_acc: 0.7744
Epoch 61/200
25800/25800 [==============================] - 19s 724us/step - loss: 4.3120e-04 - acc: 0.7219 - val_loss: 4.5584e-04 - val_acc: 0.5450
Epoch 62/200
25800/25800 [==============================] - 19s 727us/step - loss: 1.8008e-04 - acc: 0.7076 - val_loss:

25800/25800 [==============================] - 19s 729us/step - loss: 2.3672e-04 - acc: 0.8451 - val_loss: 5.8989e-08 - val_acc: 0.9641
Epoch 112/200
25800/25800 [==============================] - 19s 727us/step - loss: 1.9967e-04 - acc: 0.8641 - val_loss: 3.1889e-11 - val_acc: 0.9744
Epoch 113/200
25800/25800 [==============================] - 19s 745us/step - loss: 2.5136e-04 - acc: 0.8650 - val_loss: 7.3232e-09 - val_acc: 0.9845
Epoch 114/200
25800/25800 [==============================] - 19s 744us/step - loss: 2.4620e-04 - acc: 0.8476 - val_loss: 3.9304e-07 - val_acc: 0.8686
Epoch 115/200
25800/25800 [==============================] - 19s 743us/step - loss: 2.6180e-04 - acc: 0.8391 - val_loss: 2.0453e-08 - val_acc: 0.9738
Epoch 116/200
25800/25800 [==============================] - 19s 742us/step - loss: 3.2666e-04 - acc: 0.8556 - val_loss: 8.1108e-07 - val_acc: 0.9928
Epoch 117/200
25800/25800 [==============================] - 19s 746us/step - loss: 1.7281e-04 - acc: 0.8757 - val

25800/25800 [==============================] - 19s 749us/step - loss: 3.1632e-04 - acc: 0.9241 - val_loss: 3.3034e-06 - val_acc: 0.9938
Epoch 166/200
25800/25800 [==============================] - 19s 750us/step - loss: 1.9668e-04 - acc: 0.9217 - val_loss: 1.4140e-13 - val_acc: 0.9996
Epoch 167/200
25800/25800 [==============================] - 19s 750us/step - loss: 1.1533e-04 - acc: 0.9233 - val_loss: 2.5621e-12 - val_acc: 0.9948
Epoch 168/200
25800/25800 [==============================] - 19s 748us/step - loss: 2.9674e-04 - acc: 0.9102 - val_loss: 8.9739e-11 - val_acc: 0.9913
Epoch 169/200
25800/25800 [==============================] - 19s 751us/step - loss: 1.8968e-04 - acc: 0.9109 - val_loss: 1.1495e-10 - val_acc: 0.9975
Epoch 170/200
25800/25800 [==============================] - 19s 745us/step - loss: 1.4965e-04 - acc: 0.9219 - val_loss: 1.8195e-12 - val_acc: 0.9901
Epoch 171/200
25800/25800 [==============================] - 19s 752us/step - loss: 1.1572e-04 - acc: 0.9333 - val

In [22]:
model_yaml = classifier_E.to_yaml()

In [23]:
with open('model.yaml', 'w') as yaml_file:
    yaml_file.write(model_yaml)

In [76]:
def predict(data, model_path, weight_path):
    yaml_file = open(model_path, 'r')
    loaded = yaml_file.read()
    yaml_file.close()
    classifier = keras.models.model_from_yaml(loaded)
    classifier.load_weights(weight_path)
    result = classifier.predict(data)[0].tolist()
    output = 'C'
    possible = 'possible notes: '
    for i in range(len(result)):
        if result[i] < 0.01:
            result[i] = 0
        elif result[i] > 0.99:
            result[i] = 1
            output += '_{}'.format(i+60)
        else:
            possible += '_{}__{}'.format(i+60, result[i])
    print(output)
    print(possible)
    return result

In [51]:
data = pd.read_csv(path + 'database_{}.csv'.format(432), dtype=str)
array = data.values
array = list(array)
sample = array[34][1:11665].tolist()

In [59]:
data = []
for x in sample:
    if type(x) == str:
        x = "".join(x.split())
        data.append(float(x))
    elif type(x) == float:
        data.append(x)

In [60]:
data = np.array(data)
data = data.reshape(1, 108, 108, 1)

In [77]:
result = predict(data, 'model.yaml', "weight-25800-CNN_E_good.hdf5")

C_67_70_78_82
possible notes: 
